In [1]:
# Загрузить библиотеки
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K

import tensorflow as tf
import datetime

Using TensorFlow backend.


In [2]:
# Сделать значение цветового канала первым
#K.set_image_data_format("channels_first")

# Задать информацию об изображении
channels = 1
height = 28
width = 28

In [3]:
# Загрузить данные и цель из набора данных MNIST рукописных цифр  
(data_train, target_train), (data_test, target_test) = mnist.load_data()

# # Реформировать тренировочные данные об изображениях в признаки
data_train = data_train.reshape(data_train.shape[0], height, width, channels)

# # Реформировать тестовые данные об изображениях в признаки
data_test = data_test.reshape(data_test.shape[0], height, width, channels)

# Прошкалировать пиксельную интенсивность в диапазон между 0 и 1
features_train = data_train / 255
features_test = data_test / 255

# Преобразовать цель в кодировку с одним активным состоянием
target_train = np_utils.to_categorical(target_train)
target_test = np_utils.to_categorical(target_test)
number_of_classes = target_test.shape[1]

In [4]:
# Инициализировать нейронную сеть
network = Sequential()

# Добавить сверточный слой с 64 фильтрами, окном 5x5 и 
# активационной функций ReLU
network.add(Conv2D(filters=64,
                   kernel_size=(5, 5),
                   input_shape=(width, height, channels),
                   activation='relu'))

# Добавить максимально редуцирующий слой с окном 2x2
network.add(MaxPooling2D(pool_size=(2, 2)))

# Добавить отсеивающий слой
network.add(Dropout(0.3))

# Добавить слой для сглаживания входа
network.add(Flatten())

# Добавить полносвязный слой из 128 блоков с
# активационной функций ReLU
network.add(Dense(128, activation="relu"))

# Добавить отсеивающий слой
network.add(Dropout(0.5))

# Добавить полносвязный слой c
# активационной функцией softmax
network.add(Dense(number_of_classes, activation="softmax"))

In [5]:
#from logger import keras_metric
from logger.logger import CSVLogger
from logger.logger import TBLogger
from logger.tf import LoggerCallback

In [6]:
# CSV логгирование
#logger = CSVLogger(file_path='Loss_tabl', metric_key_list=['Loss', 'Accuracy'])

# Tensorboard логгирование
logger = TBLogger(dir_path='TB_metric', metric_key_list =['Loss', 'Accuracy'])


logger_callback = LoggerCallback(logger=logger, field_name_mapping={'Loss': 'loss', 'Accuracy': 'accuracy'})

In [7]:
# Скомплировать нейронную сеть
network.compile(
    loss="categorical_crossentropy", # Перекрестная энтропия
    optimizer="rmsprop",  # Распространение СКО
    metrics=["accuracy"]) # Точностный показатель результативности

# Натренировать нейронную сеть
history = network.fit(
    features_train,  # Признаки
    target_train,    # Цель
    epochs=7,        # Количество эпох
    verbose=1,       # Печатать описание после каждой эпохи
    batch_size=1000, # Количество наблюдений на пакет
    validation_data=(features_test, target_test), # Данные для оценивания
    callbacks=[logger_callback]) 

Epoch 1/7
60/60 [==============================] - 5s 53ms/step - loss: 1.0054 - accuracy: 0.6716 - val_loss: 0.1770 - val_accuracy: 0.9497
Epoch 2/7
60/60 [==============================] - 2s 41ms/step - loss: 0.2203 - accuracy: 0.9355 - val_loss: 0.0820 - val_accuracy: 0.9739
Epoch 3/7
60/60 [==============================] - 2s 41ms/step - loss: 0.1295 - accuracy: 0.9617 - val_loss: 0.0580 - val_accuracy: 0.9806
Epoch 4/7
60/60 [==============================] - 2s 41ms/step - loss: 0.0954 - accuracy: 0.9718 - val_loss: 0.0472 - val_accuracy: 0.9834
Epoch 5/7
60/60 [==============================] - 2s 41ms/step - loss: 0.0782 - accuracy: 0.9764 - val_loss: 0.0401 - val_accuracy: 0.9861
Epoch 6/7
60/60 [==============================] - 2s 41ms/step - loss: 0.0657 - accuracy: 0.9805 - val_loss: 0.0378 - val_accuracy: 0.9867
Epoch 7/7
60/60 [==============================] - 2s 41ms/step - loss: 0.0596 - accuracy: 0.9819 - val_loss: 0.0354 - val_accuracy: 0.9881


In [8]:
%tensorboard --logdir='TB_metric'

UsageError: Line magic function `%tensorboard` not found.
